In [34]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale
from sklearn.cross_validation import train_test_split

In [31]:
ms = pd.read_csv("/Users/thomascurran/Desktop/Data/ms_scaled.csv").drop("Unnamed: 0", axis = 1)

In [44]:
ms_quant = ms.select_dtypes(include = ["int","float"])

ms_y = ms_quant.IndexScore

ms_x = ms_quant.drop('IndexScore', axis = 1)

ms_x.Year = ms_x.Year.astype("category")


In [45]:
#scaling the index score
ms_y = scale(ms_y, with_mean = True, copy = False)

In [46]:
#split into train, test
xtrain, xtest, ytrain, ytest = train_test_split(ms_x, ms_y, train_size = .75, random_state = 1)

In [55]:
#split the training model even futher for the validation set
xtrain, xvalidation, ytrain, yvalidation = train_test_split(xtrain, ytrain, test_size = .2, random_state = 1 )

In [62]:
#define the base model
#this will be the function that is the actual NN and the one that can be tweaked

def baseline_model():
    model = Sequential()
    model.add(Dense(13,input_dim = xtrain.shape[1], kernel_initializer = 'normal',activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    #Compile Model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return(model)

In [63]:
seed = 7
np.random.seed(seed)

#evaluate model with standardized dataset:
estimator = KerasRegressor(build_fn = baseline_model, epochs = 100, batch_size = 5, verbose = 1)

In [64]:
kfold = KFold(n_splits = 10, random_state=seed)
results = cross_val_score(estimator, ms_x, ms_y, cv = kfold)
print("Mean Squared Error: %f (%f)" % (results.mean(), results.std()))

Epoch 1/100
801/801 [==============================] - 0s 457us/step - loss: 1.0912
Epoch 2/100
801/801 [==============================] - 0s 278us/step - loss: 0.2262
Epoch 3/100
801/801 [==============================] - 0s 276us/step - loss: 0.1329
Epoch 4/100
801/801 [==============================] - 0s 287us/step - loss: 0.0992
Epoch 5/100
801/801 [==============================] - 0s 278us/step - loss: 0.0822
Epoch 6/100
801/801 [==============================] - 0s 269us/step - loss: 0.0684
Epoch 7/100
801/801 [==============================] - 0s 282us/step - loss: 0.0680